### Импортируем библиотеки

In [5]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

# Описание датасета

In [11]:
data = pd.read_csv('cookie_cats.csv')

data.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [12]:
data.shape

(90189, 5)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


#### Проверка на пропуски

In [17]:
data.isna().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

Пропусков данных нет.

#### Проверка на дубликаты

In [22]:
data[data.duplicated('userid') == True]

,userid,version,sum_gamerounds,retention_1,retention_7


Дубликатов по id нет.

#### Проверка корректности сплитования

In [29]:
data_groups = data.groupby('userid', as_index=False).agg({'version': pd.Series.nunique})
data_groups[data_groups['version'] > 1]

,userid,version


Ни один юзер не попал сразу и в контроль, и в тест.

In [34]:
print(f'Размер выборки для контрольной группы: {data[data["version"] == "gate_30"].shape[0]}')
print(f'Размер выборки для тестовой группы: {data[data["version"] == "gate_40"].shape[0]}')

Размер выборки для контрольной группы: 44700
Размер выборки для тестовой группы: 45489


Все юзеры корректно распределены по 2 группам.

# Разведочный анализ

### Контроль

In [39]:
print(f"{data[data['version'] == 'gate_30'].describe()}")
print(f"Сумма retention_1 для контрольной группы = {data['retention_1'][data['version'] == 'gate_30'].sum()}")
print(f"Сумма retention_7 для контрольной группы {data['retention_7'][data['version'] == 'gate_30'].sum()}")

             userid  sum_gamerounds
count  4.470000e+04    44700.000000
mean   4.987564e+06       52.456264
std    2.881026e+06      256.716423
min    1.160000e+02        0.000000
25%    2.505469e+06        5.000000
50%    4.983631e+06       17.000000
75%    7.481497e+06       50.000000
max    9.999710e+06    49854.000000
Сумма retention_1 для контрольной группы = 20034
Сумма retention_7 для контрольной группы 8502


### Тест

In [40]:
print(f"{data[data['version'] == 'gate_40'].describe()}")
print(f"Сумма retention_1 для тестовой группы = {data['retention_1'][data['version'] == 'gate_40'].sum()}")
print(f"Сумма retention_7 для тестовой группы {data['retention_7'][data['version'] == 'gate_40'].sum()}")

             userid  sum_gamerounds
count  4.548900e+04    45489.000000
mean   5.009073e+06       51.298776
std    2.885496e+06      103.294416
min    3.770000e+02        0.000000
25%    2.517171e+06        5.000000
50%    5.007329e+06       16.000000
75%    7.510762e+06       52.000000
max    9.999861e+06     2640.000000
Сумма retention_1 для тестовой группы = 20119
Сумма retention_7 для тестовой группы 8279


# Проверка данных на нормальность

In [41]:
import plotly.express as px
from scipy.stats import kstest, shapiro

Есть смысл проверять 

### Графический метод